In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, lit, when

# Create Spark session
spark = SparkSession.builder.appName("Project").getOrCreate()
print("Spark Session Created Successfully!")

# Read CSV file with header
df = spark.read.options(header='True').csv("OfficeData.csv")
print("Basic DataFrame loaded:")
df.show()

Spark Session Created Successfully!
Basic DataFrame loaded:
+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [ ]:
# print total number of employees
total_employees = df.count()
print("Total number of employees:", total_employees)
# print the total number of departments
total_departments = df.select("Department").distinct().count()
print("Total number of departments:", total_departments)
# print the department names
department_names = df.select("Department").distinct().collect()
print("Department names:")
for dept in department_names:
    print("-", dept["Department"])
# print the total number of employees in each department
employees_per_department = df.groupBy("Department").count()
print("Total number of employees in each department:")
employees_per_department.show()
# print the total number of employees in each state
employees_per_state = df.groupBy("State").count()
print("Total number of employees in each state:")
employees_per_state.show()
# print the total number of employees in each state in each department
employees_per_state_dept = df.groupBy("State", "Department").count()
print("Total number of employees in each state in each department:")
employees_per_state_dept.show()
# print the min and max salary in each department and sort in ascending order
salary_stats_per_department = df.groupBy("Department").agg({"Salary": "min", "Salary": "max"}).orderBy("Department")
print("Min and Max salary in each department (sorted ascending):")
salary_stats_per_department.show()
# print the names of employees working in NY state under Finance department whose bonus is greater than the average bonused of employeed in NY state
avg_bonus_ny = df.filter((col("State") == "NY") & (col("Department") == "Finance")).agg({"Bonus": "avg"}).collect()[0][0]
employees_ny_finance_above_avg_bonus = df.filter((col("State") == "NY") & (col("Department") == "Finance") & (col("Bonus") > avg_bonus_ny)).select("employee_name")
print("Employees in NY Finance department with bonus greater than average:")
employees_ny_finance_above_avg_bonus.show()
# raise the salaries $500 of all employees whose age is > 45
df = df.withColumn("Salary", when(col("Age") > 45, col("Salary") + 500).otherwise(col("Salary")))
print("Salaries updated for employees older than 45.")


Total number of employees: 9
Total number of departments: 3
Total number of departments: 3
Department names:
- Sales
- Finance
- Marketing
Total number of employees in each department:
Department names:
- Sales
- Finance
- Marketing
Total number of employees in each department:
+----------+-----+
|Department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+

Total number of employees in each state:
+----------+-----+
|Department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+

Total number of employees in each state:
+-----+-----+
|State|count|
+-----+-----+
|   CA|    4|
|   NY|    5|
+-----+-----+

Total number of employees in each state in each department:
+-----+-----+
|State|count|
+-----+-----+
|   CA|    4|
|   NY|    5|
+-----+-----+

Total number of employees in each state in each department:
+-----+----------+-----+
|State|Department|count|
+-----+----------+-----+
|   CA

PermissionError: [Errno 13] Permission denied: 'employees_above_45.csv'

In [14]:
# create DF for all employeed whose age is > 45 and save it as CSV file
df_above_45 = df.filter(col("Age") > 45)
# Show the filtered data
print("Employees above 45:")
df_above_45.show()
# Save as CSV - using coalesce to create a single file
try:
    import os
    from datetime import datetime
    # Create output filename with timestamp to avoid conflicts
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"employees_above_45_{timestamp}.csv"
    df_above_45.toPandas().to_csv(output_file, index=False)
    print(f"Data saved successfully to {output_file}")
except Exception as e:
    print(f"Error saving file: {e}")
    print("Please close the file if it's open in another program and try again.")

Employees above 45:
+-------------+----------+-----+------+---+-----+
|employee_name|department|state|Salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|      Michael|     Sales|   NY| 88000| 56|20000|
|          Jen|   Finance|   NY| 81000| 53|15000|
|        Kumar| Marketing|   NY| 93000| 50|21000|
+-------------+----------+-----+------+---+-----+

Data saved successfully to employees_above_45_20251028_130807.csv
+-------------+----------+-----+------+---+-----+
|employee_name|department|state|Salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|      Michael|     Sales|   NY| 88000| 56|20000|
|          Jen|   Finance|   NY| 81000| 53|15000|
|        Kumar| Marketing|   NY| 93000| 50|21000|
+-------------+----------+-----+------+---+-----+

Data saved successfully to employees_above_45_20251028_130807.csv
